In [1]:
import fundamentalanalysis as fa
import fa_mods as famods
import pickle
from urllib.request import urlopen
import certifi
import json
import pandas as pd
from datetime import datetime
import json
import pandas as pd


with open('/home/craigc/.keys/financialmodelingprep.txt', 'r') as file:
    api_key = file.read()

In [2]:
all_companies = fa.available_companies(api_key)
lse_companies = all_companies[(all_companies.exchangeShortName == 'LSE')]
print(f"num LSE companies: {len(lse_companies)}")
lse_stocks = lse_companies[(all_companies.type == 'stock')].reset_index()
lse_stocks = lse_stocks.drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
print(f"num LSE stocks: {len(lse_stocks)}")
lse_stocks

num LSE companies: 6335
num LSE stocks: 3877


/tmp/ipykernel_374280/1542461570.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lse_stocks = lse_companies[(all_companies.type == 'stock')].reset_index()


,symbol,name
0,AIR3.L,Leverage Shares 3x Long Airbus (AIR.PA) ETC
1,KINO.L,Kinovo plc
2,OSI.L,Osirium Technologies PLC
3,VOD.L,Vodafone Group Public Limited Company
4,SRT.L,SRT Marine Systems plc
...,...,...
3872,IDEA.L,Ideagen plc
3873,MVI.L,Marwyn Value Investors Limited
3874,0KD1.L,"Tubacex, S.A."
3875,DOCS.L,Dr. Martens plc


In [3]:
with open('../data/all_lse_stock_names_221219.pkl', 'wb') as file:
    pickle.dump(lse_stocks, file)

In [4]:
with open('../data/all_lse_stock_names_221219.pkl', 'rb') as file:
    lse_stocks = pickle.load(file)

print(type(lse_stocks))
print(len(lse_stocks))
print(lse_stocks)

<class 'pandas.core.frame.DataFrame'>
3877
      symbol                                         name
0     AIR3.L  Leverage Shares 3x Long Airbus (AIR.PA) ETC
1     KINO.L                                   Kinovo plc
2      OSI.L                     Osirium Technologies PLC
3      VOD.L        Vodafone Group Public Limited Company
4      SRT.L                       SRT Marine Systems plc
...      ...                                          ...
3872  IDEA.L                                  Ideagen plc
3873   MVI.L               Marwyn Value Investors Limited
3874  0KD1.L                                Tubacex, S.A.
3875  DOCS.L                              Dr. Martens plc
3876   BKY.L                     Berkeley Energia Limited

[3877 rows x 2 columns]


In [5]:
def fmp_request(api: str, ticker: str, api_key, v4: bool=False):
    '''
    Sends an API request to financialmodelingprep.com, converting the response to a df

        Parameters:
            api (str):     The API to query, e.g. 'income-statement'
            ticker (str):  The tickername to query, e.g 'MSFT'
            api_key (str): Your fmp API key

        Returns:
            dataframe
    '''
    if v4:
        url = f'https://financialmodelingprep.com/api/v4/{api}?symbol={ticker}&apikey={api_key}'
    else:
        url = f"https://financialmodelingprep.com/api/v3/{api}/{ticker}?apikey={api_key}"
    return pd.read_json(url)

In [14]:
def get_ticker_data(ticker: str, api_key: str):
    data = {}
    data['timestamp'] = datetime.timestamp(datetime.now())
    data['income_statement'] = fa.income_statement(ticker, api_key, period="annual").drop(labels=['calendarYear', 'period', 'link', 'finalLink'], axis=0)
    data['income_statement_growth'] = famods.income_statement_growth(ticker, api_key)
    data['cashflow_statement'] = fa.cash_flow_statement(ticker, api_key, period="annual").drop(labels=['calendarYear', 'period', 'link', 'finalLink'], axis=0)
    data['cashflow_statement_growth'] = famods.cash_flow_statement_growth(ticker, api_key)
    data['balance_sheet_statement'] = fa.balance_sheet_statement(ticker, api_key, period="annual").drop(labels=['calendarYear', 'period', 'link', 'finalLink'], axis=0)
    data['balance_sheet_statement_growth'] = famods.balance_sheet_statement_growth(ticker, api_key)
    data['discounted_cash_flow'] = famods.discounted_cash_flow(ticker, api_key)
    data['advanced_discounted_cash_flow'] = famods.advanced_discounted_cash_flow(ticker, api_key)
    data['advanced_levered_discounted_cash_flow'] = famods.advanced_levered_discounted_cash_flow(ticker, api_key)
    data['key_metrics'] = fa.key_metrics(ticker, api_key, period="annual").drop(['period'])
    data['financial_ratios'] = fa.financial_ratios(ticker, api_key, period="annual").drop(['period'])
    data['growth'] = fa.financial_statement_growth(ticker, api_key, period="annual").drop(['period'])
    data['owner_earnings'] = famods.owner_earnings(ticker, api_key)
    data['enterprise_values'] = famods.enterprise_values(ticker, api_key)

    try:
        data['dividends'] = fa.stock_dividend(ticker, api_key).drop(['recordDate', 'declarationDate'], axis=1)
    except:
        pass

    try:
        data['scores'] = famods.score(ticker, api_key)
    except:
        pass


    # try:
    #     data['income_statement'].drop(labels=['calendarYear', 'period', 'link', 'finalLink'], axis=0)
    #     data['cashflow_statement'].drop(labels=['calendarYear', 'period', 'link', 'finalLink'], axis=0)
    #     data['balance_sheet_statement'].drop(labels=['calendarYear', 'period', 'link', 'finalLink'], axis=0)
    #     data['key_metrics'].drop(['period'])
    #     data['financial_ratios'].drop(['period'])
    #     data['growth'].drop(['period'])
    #     data['dividends'].drop(['recordDate', 'declarationDate'], axis=1)
    # except:
    #     pass

    return data


In [7]:
ticker = 'POLY.L'
display(fmp_request('discounted-cash-flow', ticker, api_key))
display(fmp_request('advanced_discounted_cash_flow', ticker, api_key, v4=True))
display(fmp_request('advanced_levered_discounted_cash_flow', ticker, api_key, v4=True))
display(fmp_request('historical-discounted-cash-flow-statement', ticker, api_key))

,symbol,date,dcf,Stock Price
0,POLY.L,2022-12-19,248.434826,247.55


,year,symbol,revenue,revenuePercentage,ebitda,ebitdaPercentage,ebit,ebitPercentage,depreciation,depreciationPercentage,...,ufcf,sumPvUfcf,longTermGrowthRate,terminalValue,presentTerminalValue,enterpriseValue,netDebt,equityValue,equityValuePerShare,freeCashFlowT1
0,2026,POLY.L,5.447790e+09,13.52,2.504060e+09,45.96,1.979260e+09,36.33,524800000.0,45.96,...,774050000.0,2502430000,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
1,2025,POLY.L,4.799050e+09,13.52,2.205880e+09,45.96,1.743570e+09,36.33,462310000.0,45.96,...,681880000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
2,2024,POLY.L,4.227570e+09,13.52,1.943200e+09,45.96,1.535940e+09,36.33,407260000.0,45.96,...,600680000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
3,2023,POLY.L,3.724150e+09,13.52,1.711800e+09,45.96,1.353040e+09,36.33,358760000.0,45.96,...,529150000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
4,2022,POLY.L,3.280670e+09,13.52,1.507950e+09,45.96,1.191910e+09,36.33,316040000.0,45.96,...,429980000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
5,2021,POLY.L,2.890000e+09,0.87,1.441000e+09,49.86,1.227000e+09,42.46,214000000.0,49.86,...,322390000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
6,2020,POLY.L,2.865000e+09,27.84,1.682000e+09,58.71,1.475000e+09,51.48,207000000.0,58.71,...,763110000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
7,2019,POLY.L,2.241000e+09,31.36,9.390000e+08,41.90,6.960000e+08,31.06,243000000.0,41.90,...,241580000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
8,2018,POLY.L,1.706000e+09,-6.01,6.770000e+08,39.68,4.910000e+08,28.78,186000000.0,39.68,...,247000000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000
9,2017,POLY.L,1.815000e+09,0.00,7.200000e+08,39.67,5.060000e+08,27.88,214000000.0,39.67,...,235340000.0,0,2,19937690000,14926750000,17429180000,1683000000,15746180000,33.41,789530000


,year,symbol,revenue,revenuePercentage,capitalExpenditure,capitalExpenditurePercentage,price,beta,dilutedSharesOutstanding,costofDebt,...,longTermGrowthRate,freeCashFlow,terminalValue,presentTerminalValue,enterpriseValue,netDebt,equityValue,equityValuePerShare,freeCashFlowT1,operatingCashFlowPercentage
0,2026,POLY.L,5.447790e+09,13.52,-1.169460e+09,-21.47,241,0.496,471280000.0,3.14,...,2,720370000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
1,2025,POLY.L,4.799050e+09,13.52,-1.030200e+09,-21.47,241,0.496,471280000.0,3.14,...,2,634590000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
2,2024,POLY.L,4.227570e+09,13.52,-9.075200e+08,-21.47,241,0.496,471280000.0,3.14,...,2,559020000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
3,2023,POLY.L,3.724150e+09,13.52,-7.994500e+08,-21.47,241,0.496,471280000.0,3.14,...,2,492450000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
4,2022,POLY.L,3.280670e+09,13.52,-7.042500e+08,-21.47,241,0.496,471280000.0,3.14,...,2,433810000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,34.69
5,2021,POLY.L,2.890000e+09,0.87,-7.590000e+08,-26.26,241,0.496,471280000.0,3.14,...,2,436000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,41.35
6,2020,POLY.L,2.865000e+09,27.84,-5.830000e+08,-20.35,241,0.496,471280000.0,3.14,...,2,609000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,41.61
7,2019,POLY.L,2.241000e+09,31.36,-4.360000e+08,-19.46,241,0.496,471280000.0,3.14,...,2,260000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,31.06
8,2018,POLY.L,1.706000e+09,-6.01,-3.440000e+08,-20.16,241,0.496,471280000.0,3.14,...,2,169000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,30.07
9,2017,POLY.L,1.815000e+09,0.00,-3.830000e+08,-21.10,241,0.496,471280000.0,3.14,...,2,150000000.0,18554940000,13891520000,16252160000,1683000000,1.456916e+10,30.91,734780000,29.37


,symbol,date,price,dcf
0,POLY.L,2021-12-31,1048.5,11.37
1,POLY.L,2020-12-31,1581.0,16.63
2,POLY.L,2019-12-31,1266.0,13.20
3,POLY.L,2018-12-31,857.6,9.42
4,POLY.L,2017-12-31,825.8,8.34
5,POLY.L,2016-12-31,898.0,9.09
6,POLY.L,2015-12-31,575.0,5.87
7,POLY.L,2014-12-31,597.5,6.37
8,POLY.L,2013-12-31,579.5,5.96
9,POLY.L,2012-12-31,1088.0,10.93


In [4]:
ticker = 'POLY.L'
data = get_ticker_data(ticker, api_key)
# display(data)

In [9]:
print(data.keys())
key = 'discounted_cash_flow'
display(data[key].keys())
display(data[key])

dict_keys(['timestamp', 'income_statement', 'income_statement_growth', 'cashflow_statement', 'cashflow_statement_growth', 'balance_sheet_statement', 'balance_sheet_statement_growth', 'discounted_cash_flow', 'advanced_discounted_cash_flow', 'advanced_levered_discounted_cash_flow', 'key_metrics', 'financial_ratios', 'growth', 'dividends', 'scores', 'owner_earnings', 'enterprise_values'])


Index(['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014',
       '2013', '2012', '2011'],
      dtype='object')

,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
date,2022-12-19,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31
dcf,248.434826,11.37,16.63,13.2,9.42,8.34,9.09,5.87,6.37,5.96,10.93,13.11
price,247.55,1048.5,1581.0,1266.0,857.6,825.8,898.0,575.0,597.5,579.5,1088.0,1131.0


In [5]:
import sys
import math

def get_mem_usage_kb(data: dict):

    mem_usage_bytes = 0
    for val in data.values():
        if isinstance(val, pd.DataFrame):
            mem_usage_bytes += val.memory_usage().sum()
    
    return math.ceil(mem_usage_bytes / 1024)

data_mem_kb = get_mem_usage_kb(data)
print(f"Data is using {get_mem_usage_kb(data)}kb")
print(f"Total memory use: {len(lse_stocks) * data_mem_kb / 1024}")

request_duration = 10.6
print(f"Total download time: {len(lse_stocks) * request_duration / 3600: .1f}h")

Data is using 55kb
Total memory use: 208.2373046875
Total download time:  11.4h


In [6]:
for key, val in data.items():
    print(key)
    display(val)

timestamp


1671492951.685807

income_statement


,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
cik,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
fillingDate,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31,2010-12-31,2009-12-31,2008-12-31,2007-12-31,2006-12-31,2005-12-31
acceptedDate,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31,2010-12-31,2009-12-31,2008-12-31,2007-12-31,2006-12-31,2005-12-31
revenue,2890000000,2865000000,2241000000,1706000000,1815000000,1583000000,1441093000,1690391000,1706597000,1854065000,1326430566.4,925375671,560737000,502731000,308747000,315596000,238973000
costOfRevenue,1307000000,1129000000,1197000000,971000000,1106000000,846000000,779228000,984045000,1277123000,889565000,625739715.8,458114425.8,284416000,300729000,254638000,171283000,136173000
grossProfit,1583000000,1736000000,1044000000,735000000,709000000,737000000,661865000,706346000,429474000,964500000,700690850.6,467261245.2,276321000,202002000,54109000,144313000,102800000
grossProfitRatio,0.547751,0.605934,0.465863,0.430832,0.390634,0.465572,0.45928,0.41786,0.251655,0.520208,0.528253,0.504942,0.492782,0.401809,0.175254,0.457271,0.430174
researchAndDevelopmentExpenses,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30211704,8105628.9,0.0,0.0,0.0,0.0,0.0
generalAndAdministrativeExpenses,31000000,23000000,25000000,29000000,28000000,23000000,30334000,30923000,33093000,30503000,0.0,0.0,2440000.0,1984000.0,48412000,28351000,22397000


income_statement_growth


...

,altmanZScore,piotroskiScore,workingCapital,totalAssets,retainedEarnings,ebit,marketCap,totalLiabilities,revenue
0,1.160701,7.0,1.188E9,7.436E9,1.252E9,6.4475E8,1.136698752E9,4.165E9,2.10725E9


owner_earnings


,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
date,2019-01-31,2018-01-31,2017-01-31,2016-01-31,2015-01-31,2014-01-31,2013-01-31,2012-01-31,2011-01-31,2010-01-31,2009-01-31,2008-01-31,2007-01-31
averagePPE,1.17,1.23,1.29,1.33,1.37,1.41,1.52,1.56,1.45,1.25,1.42,1.3,1.32
maintenanceCapex,0.0,-86000000.0,-20930000.0,-20601399.25,-51356500.0,-52437750.0,-79862000.0,0.0,0.0,0.0,-35011874.5,0.0,-28913500.0
ownersEarnings,174000000.0,42250000.0,112320000.0,111898600.75,71154500.0,76272000.0,35554750.0,123958750.0,53024726.42,53803714.8,8557875.5,20192250.0,-29615000.0
growthCapex,-156755000.0,0.0,-74820000.0,-47148600.75,0.0,0.0,0.0,-205249794.67,-145482663.31,-113949584.69,-20548625.5,-63093296.0,0.0
ownersEarningsPerShare,0.36,0.09,0.26,0.26,0.17,0.19,0.09,0.32,0.14,0.15,0.03,0.07,-0.1


enterprise_values


,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
date,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31
stockPrice,1048.5,1581.0,1266.0,857.599976,825.799988,898.0,575.0,597.5,579.5,1088.0,1131.0
numberOfShares,471278987,471278987,469926157,449016966,429880907,426135182,422958680,395924528,388307843,382705692,366969000
marketCapitalization,494136017869.5,745092078447,594926514762,385076939085.585999,354995647756.052979,382669393436,243201241000,236564905480,225024395018.5,416383792896,415041939000
minusCashAndCashEquivalents,417000000,386000000,253000000,379000000,36000000,48000000,51798000,157224000,65567000,18622000,658795422.8
addTotalDebt,2100000000,1770000000,1764000000,1899000000,1456000000,1378000000,1349546000,1322635000,1111144000,863823000,1003095399.1
enterpriseValue,6624360178.695,8834920784.470001,7460265147.62,5370769390.85586,4969956477.56053,5156693934.360001,3729760410,3531060054.8,3295820950.185,5009038928.960001,4494719366.3


In [12]:
# print('timestamp')
# display(data['timestamp'])
# print('income_statement')
# display(data['income_statement'])
# print('income_statement_growth')
# display(data['income_statement_growth'])
# print('cashflow_statement')
# display(data['cashflow_statement'])
# print('cashflow_statement_growth')
# display(data['cashflow_statement_growth'])
# print('balance_sheet_statement')
# display(data['balance_sheet_statement'])
# print('balance_sheet_statement_growth')
# display(data['balance_sheet_statement_growth'])
# print('discounted_cash_flow')
# display(data['discounted_cash_flow'])
# print('advanced_discounted_cash_flow')
# display(data['advanced_discounted_cash_flow'])
# print('advanced_levered_discounted_cash_flow')
# display(data['advanced_levered_discounted_cash_flow'])
# print('key_metrics')
# display(data['key_metrics'])
# print('financial_ratios')
# display(data['financial_ratios'])
# print('growth')
# display(data['growth'])
# print('dividends')
# display(data['dividends']) # aggregate & rotate?
# print('scores')
# display(data['scores'])
# print('enterprise_values')
# display(data['enterprise_values'])

In [ ]:
num_lse_stocks = len(lse_stocks)

lse_stock_data = {}

failed = []

In [20]:
for index, row in lse_stocks.iterrows():
    symbol = row['symbol']
    print(f"{index} of {num_lse_stocks}: {symbol} ")

    if symbol in lse_stock_data:
        print(f"Already have {symbol}, skipping")
        continue

    try:
        data = get_ticker_data(symbol, api_key)
        lse_stock_data[symbol] = data
    except:
        print(f"{index} of {num_lse_stocks}: {symbol} FAILED")
        failed.append(symbol)

0 of 3877: AIR3.L 
0 of 3877: AIR3.L FAILED
1 of 3877: KINO.L 
Already have KINO.L, skipping
2 of 3877: OSI.L 
Already have OSI.L, skipping
3 of 3877: VOD.L 
Already have VOD.L, skipping
4 of 3877: SRT.L 
Already have SRT.L, skipping
5 of 3877: YGEN.L 
Already have YGEN.L, skipping
6 of 3877: PLTS.L 
6 of 3877: PLTS.L FAILED
7 of 3877: 72HN.L 
7 of 3877: 72HN.L FAILED
8 of 3877: FAS.L 
9 of 3877: BGLF.L 
9 of 3877: BGLF.L FAILED
10 of 3877: OAP3.L 
11 of 3877: CNIC.L 
12 of 3877: CRDA.L 
13 of 3877: SMIN.L 
14 of 3877: 0LE3.L 
14 of 3877: 0LE3.L FAILED
15 of 3877: 0L7F.L 
15 of 3877: 0L7F.L FAILED
16 of 3877: UL3S.L 
16 of 3877: UL3S.L FAILED
17 of 3877: 0RJ6.L 
17 of 3877: 0RJ6.L FAILED
18 of 3877: 0HCZ.L 
18 of 3877: 0HCZ.L FAILED
19 of 3877: PYP3.L 
19 of 3877: PYP3.L FAILED
20 of 3877: MSFS.L 
20 of 3877: MSFS.L FAILED
21 of 3877: NVDS.L 
21 of 3877: NVDS.L FAILED
22 of 3877: CO2.L 
22 of 3877: CO2.L FAILED
23 of 3877: 0YC0.L 
23 of 3877: 0YC0.L FAILED
24 of 3877: 3BA.L 
24 of 3877

In [17]:
symbol = 'SRT.L'
get_ticker_data(symbol, api_key)
# fa.income_statement(symbol, api_key, period="annual")

{'timestamp': 1671493349.401306,
 'income_statement':                                                2022        2021        2020  \
 reportedCurrency                                GBP         GBP         GBP   
 cik                                            None        None        None   
 fillingDate                              2022-03-31  2021-03-31  2020-03-31   
 acceptedDate                             2022-03-31  2021-03-31  2020-03-31   
 revenue                                   8172900.0   8275022.0    18908062   
 costOfRevenue                             5500942.0   5097419.0    14537092   
 grossProfit                               2671958.0   3177603.0   4370970.0   
 grossProfitRatio                           0.326929    0.383999     0.23117   
 researchAndDevelopmentExpenses                  0.0         0.0         0.0   
 generalAndAdministrativeExpenses          8721560.0   8048640.0   6883261.0   
 sellingAndMarketingExpenses                     0.0         0.0   

In [23]:
print(len(failed))
print(len(lse_stock_data))

2428
1451


{'timestamp': 1671498468.864519,
 'income_statement':                                                 2022         2021  \
 reportedCurrency                                 GBP          GBP   
 cik                                             None         None   
 fillingDate                               2022-09-30   2021-09-30   
 acceptedDate                              2022-09-30   2021-09-30   
 revenue                                  32551000000  32791000000   
 costOfRevenue                            26513000000  26764000000   
 grossProfit                               6038000000   6027000000   
 grossProfitRatio                            0.185494       0.1838   
 researchAndDevelopmentExpenses                   0.0          0.0   
 generalAndAdministrativeExpenses           323000000    336000000   
 sellingAndMarketingExpenses               2021000000   2118000000   
 sellingGeneralAndAdministrativeExpenses   2344000000   2454000000   
 otherExpenses                       

In [24]:
with open('../data/all_lse_stock_data_221219.pkl', 'wb') as file:
    pickle.dump(lse_stock_data, file)

with open('../data/all_lse_stock_data_221219_failed.pkl', 'wb') as file:
    pickle.dump(failed, file)

In [25]:
with open('../data/all_lse_stock_data_221219.pkl', 'rb') as file:
    lse_stock_data_2 = pickle.load(file)

len(lse_stock_data_2)

1451

In [26]:
lse_stock_data['POLY.L']

{'timestamp': 1671518306.229115,
 'income_statement':                                                2021        2020        2019  \
 reportedCurrency                                USD         USD         USD   
 cik                                            None        None        None   
 fillingDate                              2021-12-31  2020-12-31  2019-12-31   
 acceptedDate                             2021-12-31  2020-12-31  2019-12-31   
 revenue                                  2890000000  2865000000  2241000000   
 costOfRevenue                            1307000000  1129000000  1197000000   
 grossProfit                              1583000000  1736000000  1044000000   
 grossProfitRatio                           0.547751    0.605934    0.465863   
 researchAndDevelopmentExpenses                  0.0         0.0         0.0   
 generalAndAdministrativeExpenses           31000000    23000000    25000000   
 sellingAndMarketingExpenses                     0.0         0.0   